In [1]:
import pandas as pd
import numpy as np
import pymongo
from pymongo import MongoClient
import sqlite3
import mysql.connector
import pytz
import datetime

In [2]:
df_store_status=pd.read_csv('store_status.csv')
df_timezoned=pd.read_csv('timezoned.csv')
df_Menu_hours=pd.read_csv('Menu_hours.csv')

In [3]:
conn = sqlite3.connect('user3.db')

In [4]:
df_store_status.to_sql('df_store_status', conn, if_exists='replace', index=False)
df_timezoned.to_sql('df_timezoned', conn, if_exists='replace', index=False)
df_Menu_hours.to_sql('df_Menu_hours', conn, if_exists='replace', index=False)

86198

In [5]:
dt_utc=datetime.datetime.now(pytz.utc)
print(dt_utc)
dt_dynamic=datetime.datetime.now(pytz.timezone('America/New_York'))
print(dt_dynamic)

2023-07-04 12:48:50.655353+00:00
2023-07-04 08:48:50.700868-04:00


In [6]:
int(dt_dynamic.strftime('%z')[:3])

-4

In [7]:
time_diff=[]
for i in range(len(df_timezoned)):
    dt_dynamic=datetime.datetime.now(pytz.timezone(df_timezoned['timezone_str'][i]))
    time_diff.append(int(dt_dynamic.strftime('%z')[:3]))

In [8]:
# time_diff=[]
# for i in range(len(df_timezoned)):
#     dt_dynamic=datetime.datetime.now(pytz.timezone(df_timezoned['timezone_str'][i]))
#     time_diff.append(int(dt_dynamic.strftime('%z')[:3]))

In [9]:
df_timezoned['time_diff']=time_diff

In [10]:
df_timezoned.head()

,store_id,timezone_str,time_diff
0,8139926242460185114,Asia/Beirut,3
1,5415949628544298339,America/Boise,-6
2,3408529570017053440,America/Denver,-6
3,9055649751952768824,America/Denver,-6
4,4428372089193592098,America/Denver,-6


In [11]:
df_store_status.shape

(1822080, 3)

In [12]:
query = ''' 
        SELECT df_store_status.store_id, df_store_status.status, df_store_status.timestamp_utc, COALESCE(df_timezoned.timezone_str, 'America/Chicago') AS timezone_str, 
        substr(df_store_status.timestamp_utc,1,10) as Date,
        (((strftime('%w', substr(df_store_status.timestamp_utc, 1, 10)) ) ))
        AS Day
        FROM df_store_status
        LEFT JOIN df_timezoned
        ON df_store_status.store_id = df_timezoned.store_id
''' 
df_id = pd.read_sql_query(query, conn)
df_id.shape

(1822080, 6)

In [13]:
df_id['Day'].value_counts()

Day
3    452542
4    358889
5    309060
2    300997
6    288920
0    111672
Name: count, dtype: int64

In [14]:
df_id.head()

,store_id,status,timestamp_utc,timezone_str,Date,Day
0,8419537941919820732,active,2023-01-22 12:09:39.388884 UTC,America/Chicago,2023-01-22,0
1,54515546588432327,active,2023-01-24 09:06:42.605777 UTC,America/New_York,2023-01-24,2
2,8377465688456570187,active,2023-01-24 09:07:26.441407 UTC,America/Los_Angeles,2023-01-24,2
3,5955337179846162144,active,2023-01-24 09:08:07.634899 UTC,America/New_York,2023-01-24,2
4,1169347689335808384,active,2023-01-24 09:08:18.436854 UTC,America/Los_Angeles,2023-01-24,2


In [15]:
df_store_status.head()

,store_id,status,timestamp_utc
0,8419537941919820732,active,2023-01-22 12:09:39.388884 UTC
1,54515546588432327,active,2023-01-24 09:06:42.605777 UTC
2,8377465688456570187,active,2023-01-24 09:07:26.441407 UTC
3,5955337179846162144,active,2023-01-24 09:08:07.634899 UTC
4,1169347689335808384,active,2023-01-24 09:08:18.436854 UTC


In [16]:
query='''
    SELECT 
    (strftime('%w', substr(df_store_status.timestamp_utc,1,10))-1) AS Day
    FROM df_store_status
'''
df_check=pd.read_sql_query(query, conn)
df_check.head()

,Day
0,-1
1,1
2,1
3,1
4,1


In [17]:
df_check['Day'].value_counts()

Day
 2    452542
 3    358889
 4    309060
 1    300997
 5    288920
-1    111672
Name: count, dtype: int64

In [18]:
query = ''' 
        SELECT df_store_status.store_id, df_store_status.status, df_store_status.timestamp_utc, COALESCE(df_timezoned.timezone_str, 'America/Chicago') AS timezone_str, 
        substr(df_store_status.timestamp_utc,1,10) as Date,
        CASE (strftime('%w', substr(df_store_status.timestamp_utc,1,10))-1)%7 
                When 0 THEN 'Monday'
                When 1 THEN 'Tuesday'
                When 2 THEN 'Wednesday'
                When 3 THEN 'Thursday'
                When 4 THEN 'Friday'
                When 5 THEN 'Saturday'
                When -1 THEN 'Sunday'
                ELSE 'Unknown'
        END AS Day
        FROM df_store_status
        LEFT JOIN df_timezoned
        ON df_store_status.store_id = df_timezoned.store_id
''' 
df_id = pd.read_sql_query(query, conn)
df_id.shape

(1822080, 6)

In [19]:
df_id['Day'].value_counts()

Day
Wednesday    452542
Thursday     358889
Friday       309060
Tuesday      300997
Saturday     288920
Sunday       111672
Name: count, dtype: int64

In [20]:
df_id.to_csv('df_id2.csv', index=False)

In [21]:
df_id.isna().sum()

store_id         0
status           0
timestamp_utc    0
timezone_str     0
Date             0
Day              0
dtype: int64

In [22]:
df_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1822080 entries, 0 to 1822079
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   store_id       int64 
 1   status         object
 2   timestamp_utc  object
 3   timezone_str   object
 4   Date           object
 5   Day            object
dtypes: int64(1), object(5)
memory usage: 83.4+ MB


In [23]:
timestamp_timezone=[]
rows_to_drop = []
i=1
for i in range(len(df_id)):
    try:
        given_datetime = datetime.datetime.strptime(df_id['timestamp_utc'][i], "%Y-%m-%d %H:%M:%S.%f %Z")
        dt_dynamic=datetime.datetime.now(pytz.timezone(df_id['timezone_str'][i]))
        time_difference=int(dt_dynamic.strftime('%z')[:3])
        time_difference=datetime.timedelta(hours=time_difference)
        result_datetime = given_datetime + time_difference
        result_timestamp = result_datetime.strftime("%Y-%m-%d %H:%M:%S.%f")
        timestamp_timezone.append(result_timestamp)
    except:
        rows_to_drop.append(i)
        print(i)
        i+=1


1051895


In [24]:
len(timestamp_timezone)

1822079

In [25]:
df_id = df_id.drop(rows_to_drop)

In [26]:
df_id['Day'].value_counts()

Day
Wednesday    452541
Thursday     358889
Friday       309060
Tuesday      300997
Saturday     288920
Sunday       111672
Name: count, dtype: int64

In [27]:
df_id.shape

(1822079, 6)

In [28]:
df_id['timestamp_timezone']=timestamp_timezone

In [29]:
df_id.head()

,store_id,status,timestamp_utc,timezone_str,Date,Day,timestamp_timezone
0,8419537941919820732,active,2023-01-22 12:09:39.388884 UTC,America/Chicago,2023-01-22,Sunday,2023-01-22 07:09:39.388884
1,54515546588432327,active,2023-01-24 09:06:42.605777 UTC,America/New_York,2023-01-24,Tuesday,2023-01-24 05:06:42.605777
2,8377465688456570187,active,2023-01-24 09:07:26.441407 UTC,America/Los_Angeles,2023-01-24,Tuesday,2023-01-24 02:07:26.441407
3,5955337179846162144,active,2023-01-24 09:08:07.634899 UTC,America/New_York,2023-01-24,Tuesday,2023-01-24 05:08:07.634899
4,1169347689335808384,active,2023-01-24 09:08:18.436854 UTC,America/Los_Angeles,2023-01-24,Tuesday,2023-01-24 02:08:18.436854


In [30]:
df_id.to_csv('df_id3.csv', index=False)

In [31]:
df_id_small=df_id[:5]

In [32]:
df_id_small

,store_id,status,timestamp_utc,timezone_str,Date,Day,timestamp_timezone
0,8419537941919820732,active,2023-01-22 12:09:39.388884 UTC,America/Chicago,2023-01-22,Sunday,2023-01-22 07:09:39.388884
1,54515546588432327,active,2023-01-24 09:06:42.605777 UTC,America/New_York,2023-01-24,Tuesday,2023-01-24 05:06:42.605777
2,8377465688456570187,active,2023-01-24 09:07:26.441407 UTC,America/Los_Angeles,2023-01-24,Tuesday,2023-01-24 02:07:26.441407
3,5955337179846162144,active,2023-01-24 09:08:07.634899 UTC,America/New_York,2023-01-24,Tuesday,2023-01-24 05:08:07.634899
4,1169347689335808384,active,2023-01-24 09:08:18.436854 UTC,America/Los_Angeles,2023-01-24,Tuesday,2023-01-24 02:08:18.436854


In [33]:
df_date=df_id

In [34]:
day_map={
    'Monday':0,
    'Tuesday':1,
    'Wednesday':2,
    'Thursday':3,
    'Friday':4,
    'Saturday':5,
    'Sunday':6
}

In [35]:
df_id_small.to_sql('df_id_small', conn, if_exists='replace', index=False)
df_id_small.columns

Index(['store_id', 'status', 'timestamp_utc', 'timezone_str', 'Date', 'Day',
       'timestamp_timezone'],
      dtype='object')

In [36]:
reverse_day_map = {v: k for k, v in day_map.items()}
reverse_day_map

{0: 'Monday',
 1: 'Tuesday',
 2: 'Wednesday',
 3: 'Thursday',
 4: 'Friday',
 5: 'Saturday',
 6: 'Sunday'}

In [37]:
df_Menu_hours=pd.read_csv('Menu_hours.csv')
df_Menu_hours['DayOfWeek']=df_Menu_hours['day'].map(reverse_day_map)
df_Menu_hours.head()

,store_id,day,start_time_local,end_time_local,DayOfWeek
0,1481966498820158979,4,00:00:00,00:10:00,Friday
1,1481966498820158979,2,00:00:00,00:10:00,Wednesday
2,1481966498820158979,0,00:00:00,00:10:00,Monday
3,1481966498820158979,1,00:00:00,00:10:00,Tuesday
4,1481966498820158979,5,00:00:00,00:10:00,Saturday


In [38]:
df_Menu_hours.to_sql('df_Menu_hours', conn, if_exists='replace', index=False)

86198

In [39]:
query = ''' 
    SELECT df_id_small.store_id, df_id_small.day, df_id_small.timestamp_utc, df_id_small.timezone_str, df_id_small.Date, df_id_small.Day, df_id_small.timestamp_timezone,
    df_Menu_hours.start_time_local, df_Menu_hours.end_time_local, df_Menu_hours.DayOfWeek
    FROM df_id_small
    LEFT JOIN df_Menu_hours
    ON df_id_small.store_id = df_Menu_hours.store_id
    WHERE df_Menu_hours.DayOfWeek = df_id_small.day
''' 
df_filter_small = pd.read_sql_query(query, conn)
df_filter_small

,store_id,Day,timestamp_utc,timezone_str,Date,Day,timestamp_timezone,start_time_local,end_time_local,DayOfWeek
0,54515546588432327,Tuesday,2023-01-24 09:06:42.605777 UTC,America/New_York,2023-01-24,Tuesday,2023-01-24 05:06:42.605777,08:00:00,18:30:00,Tuesday
1,8377465688456570187,Tuesday,2023-01-24 09:07:26.441407 UTC,America/Los_Angeles,2023-01-24,Tuesday,2023-01-24 02:07:26.441407,09:00:00,21:30:00,Tuesday
2,5955337179846162144,Tuesday,2023-01-24 09:08:07.634899 UTC,America/New_York,2023-01-24,Tuesday,2023-01-24 05:08:07.634899,11:30:00,21:30:00,Tuesday
3,1169347689335808384,Tuesday,2023-01-24 09:08:18.436854 UTC,America/Los_Angeles,2023-01-24,Tuesday,2023-01-24 02:08:18.436854,10:00:00,20:00:00,Tuesday


In [40]:
df_id.to_sql('df_id', conn, if_exists='replace', index=False)
df_id.columns

Index(['store_id', 'status', 'timestamp_utc', 'timezone_str', 'Date', 'Day',
       'timestamp_timezone'],
      dtype='object')

In [41]:
df_Menu_hours.shape

(86198, 5)

In [42]:
df_id.head()

,store_id,status,timestamp_utc,timezone_str,Date,Day,timestamp_timezone
0,8419537941919820732,active,2023-01-22 12:09:39.388884 UTC,America/Chicago,2023-01-22,Sunday,2023-01-22 07:09:39.388884
1,54515546588432327,active,2023-01-24 09:06:42.605777 UTC,America/New_York,2023-01-24,Tuesday,2023-01-24 05:06:42.605777
2,8377465688456570187,active,2023-01-24 09:07:26.441407 UTC,America/Los_Angeles,2023-01-24,Tuesday,2023-01-24 02:07:26.441407
3,5955337179846162144,active,2023-01-24 09:08:07.634899 UTC,America/New_York,2023-01-24,Tuesday,2023-01-24 05:08:07.634899
4,1169347689335808384,active,2023-01-24 09:08:18.436854 UTC,America/Los_Angeles,2023-01-24,Tuesday,2023-01-24 02:08:18.436854


In [43]:
query = ''' 
    SELECT df_id.store_id, df_id.day, df_id.status, df_id.timestamp_utc, df_id.timezone_str, df_id.Date, df_id.timestamp_timezone,
    df_Menu_hours.start_time_local, df_Menu_hours.end_time_local, df_Menu_hours.DayOfWeek
    FROM df_id
    LEFT JOIN df_Menu_hours
    ON df_id.store_id = df_Menu_hours.store_id
    WHERE df_Menu_hours.DayOfWeek = df_id.day
    AND df_id.status='active'
    AND time(df_id.timestamp_timezone) >= time(df_Menu_hours.start_time_local)
    AND time(df_id.timestamp_timezone) <= time(df_Menu_hours.end_time_local)
''' 
df_filter = pd.read_sql_query(query, conn)
df_filter

,store_id,Day,status,timestamp_utc,timezone_str,Date,timestamp_timezone,start_time_local,end_time_local,DayOfWeek
0,3739729523266121270,Tuesday,active,2023-01-24 09:08:23.138922 UTC,America/New_York,2023-01-24,2023-01-24 05:08:23.138922,00:00:00,23:59:59,Tuesday
1,4430370444829587327,Tuesday,active,2023-01-24 09:09:37.456563 UTC,America/Los_Angeles,2023-01-24,2023-01-24 02:09:37.456563,00:00:00,04:45:00,Tuesday
2,5627904252074297008,Tuesday,active,2023-01-24 14:01:52.138196 UTC,America/New_York,2023-01-24,2023-01-24 10:01:52.138196,05:30:00,23:59:59,Tuesday
3,605803287923472894,Tuesday,active,2023-01-24 05:58:22.253512 UTC,America/Los_Angeles,2023-01-24,2023-01-23 22:58:22.253512,10:00:00,23:59:59,Tuesday
4,5693932324957995427,Tuesday,active,2023-01-24 08:09:45.795033 UTC,America/Chicago,2023-01-24,2023-01-24 03:09:45.795033,00:00:00,23:59:59,Tuesday
...,...,...,...,...,...,...,...,...,...,...
645559,7738034470109474246,Friday,active,2023-01-20 16:05:52.75651 UTC,America/Chicago,2023-01-20,2023-01-20 11:05:52.756510,11:00:00,21:00:00,Friday
645560,2233199855788677277,Friday,active,2023-01-20 16:05:49.910902 UTC,America/New_York,2023-01-20,2023-01-20 12:05:49.910902,11:30:00,14:30:00,Friday
645561,5009413255373461915,Friday,active,2023-01-20 14:03:54.773505 UTC,America/New_York,2023-01-20,2023-01-20 10:03:54.773505,10:00:00,19:00:00,Friday
645562,6197071901978246272,Friday,active,2023-01-20 18:07:13.610221 UTC,America/Los_Angeles,2023-01-20,2023-01-20 11:07:13.610221,11:00:00,21:00:00,Friday


In [44]:
df_filter['Day'].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
      dtype=object)

In [45]:
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645564 entries, 0 to 645563
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   store_id            645564 non-null  int64 
 1   Day                 645564 non-null  object
 2   status              645564 non-null  object
 3   timestamp_utc       645564 non-null  object
 4   timezone_str        645564 non-null  object
 5   Date                645564 non-null  object
 6   timestamp_timezone  645564 non-null  object
 7   start_time_local    645564 non-null  object
 8   end_time_local      645564 non-null  object
 9   DayOfWeek           645564 non-null  object
dtypes: int64(1), object(9)
memory usage: 49.3+ MB


In [46]:
df_filter['timestamp_timezone']=pd.to_datetime(df_filter['timestamp_timezone'])

In [47]:
df_filter['minute']=df_filter['timestamp_timezone'].dt.minute

In [48]:
df_filter.head()

,store_id,Day,status,timestamp_utc,timezone_str,Date,timestamp_timezone,start_time_local,end_time_local,DayOfWeek,minute
0,3739729523266121270,Tuesday,active,2023-01-24 09:08:23.138922 UTC,America/New_York,2023-01-24,2023-01-24 05:08:23.138922,00:00:00,23:59:59,Tuesday,8
1,4430370444829587327,Tuesday,active,2023-01-24 09:09:37.456563 UTC,America/Los_Angeles,2023-01-24,2023-01-24 02:09:37.456563,00:00:00,04:45:00,Tuesday,9
2,5627904252074297008,Tuesday,active,2023-01-24 14:01:52.138196 UTC,America/New_York,2023-01-24,2023-01-24 10:01:52.138196,05:30:00,23:59:59,Tuesday,1
3,605803287923472894,Tuesday,active,2023-01-24 05:58:22.253512 UTC,America/Los_Angeles,2023-01-24,2023-01-23 22:58:22.253512,10:00:00,23:59:59,Tuesday,58
4,5693932324957995427,Tuesday,active,2023-01-24 08:09:45.795033 UTC,America/Chicago,2023-01-24,2023-01-24 03:09:45.795033,00:00:00,23:59:59,Tuesday,9


In [49]:
df_filter['minute']=df_filter['minute'].apply(lambda x: 60-x)

In [50]:
df_filter.rename(columns={'minute': 'uptime_last_hour'}, inplace=True)
df_filter.head()

,store_id,Day,status,timestamp_utc,timezone_str,Date,timestamp_timezone,start_time_local,end_time_local,DayOfWeek,uptime_last_hour
0,3739729523266121270,Tuesday,active,2023-01-24 09:08:23.138922 UTC,America/New_York,2023-01-24,2023-01-24 05:08:23.138922,00:00:00,23:59:59,Tuesday,52
1,4430370444829587327,Tuesday,active,2023-01-24 09:09:37.456563 UTC,America/Los_Angeles,2023-01-24,2023-01-24 02:09:37.456563,00:00:00,04:45:00,Tuesday,51
2,5627904252074297008,Tuesday,active,2023-01-24 14:01:52.138196 UTC,America/New_York,2023-01-24,2023-01-24 10:01:52.138196,05:30:00,23:59:59,Tuesday,59
3,605803287923472894,Tuesday,active,2023-01-24 05:58:22.253512 UTC,America/Los_Angeles,2023-01-24,2023-01-23 22:58:22.253512,10:00:00,23:59:59,Tuesday,2
4,5693932324957995427,Tuesday,active,2023-01-24 08:09:45.795033 UTC,America/Chicago,2023-01-24,2023-01-24 03:09:45.795033,00:00:00,23:59:59,Tuesday,51


In [51]:
df_filter['downtime_last_hour']=60-df_filter['uptime_last_hour']
df_filter.head()

,store_id,Day,status,timestamp_utc,timezone_str,Date,timestamp_timezone,start_time_local,end_time_local,DayOfWeek,uptime_last_hour,downtime_last_hour
0,3739729523266121270,Tuesday,active,2023-01-24 09:08:23.138922 UTC,America/New_York,2023-01-24,2023-01-24 05:08:23.138922,00:00:00,23:59:59,Tuesday,52,8
1,4430370444829587327,Tuesday,active,2023-01-24 09:09:37.456563 UTC,America/Los_Angeles,2023-01-24,2023-01-24 02:09:37.456563,00:00:00,04:45:00,Tuesday,51,9
2,5627904252074297008,Tuesday,active,2023-01-24 14:01:52.138196 UTC,America/New_York,2023-01-24,2023-01-24 10:01:52.138196,05:30:00,23:59:59,Tuesday,59,1
3,605803287923472894,Tuesday,active,2023-01-24 05:58:22.253512 UTC,America/Los_Angeles,2023-01-24,2023-01-23 22:58:22.253512,10:00:00,23:59:59,Tuesday,2,58
4,5693932324957995427,Tuesday,active,2023-01-24 08:09:45.795033 UTC,America/Chicago,2023-01-24,2023-01-24 03:09:45.795033,00:00:00,23:59:59,Tuesday,51,9


In [52]:
df_filter=df_filter.drop(['Day'],axis=1)
df_filter.head()

,store_id,status,timestamp_utc,timezone_str,Date,timestamp_timezone,start_time_local,end_time_local,DayOfWeek,uptime_last_hour,downtime_last_hour
0,3739729523266121270,active,2023-01-24 09:08:23.138922 UTC,America/New_York,2023-01-24,2023-01-24 05:08:23.138922,00:00:00,23:59:59,Tuesday,52,8
1,4430370444829587327,active,2023-01-24 09:09:37.456563 UTC,America/Los_Angeles,2023-01-24,2023-01-24 02:09:37.456563,00:00:00,04:45:00,Tuesday,51,9
2,5627904252074297008,active,2023-01-24 14:01:52.138196 UTC,America/New_York,2023-01-24,2023-01-24 10:01:52.138196,05:30:00,23:59:59,Tuesday,59,1
3,605803287923472894,active,2023-01-24 05:58:22.253512 UTC,America/Los_Angeles,2023-01-24,2023-01-23 22:58:22.253512,10:00:00,23:59:59,Tuesday,2,58
4,5693932324957995427,active,2023-01-24 08:09:45.795033 UTC,America/Chicago,2023-01-24,2023-01-24 03:09:45.795033,00:00:00,23:59:59,Tuesday,51,9


In [53]:
df_filter.to_sql('df_filter', conn, if_exists='replace', index=False)

645564

In [54]:
df_filter['DayOfWeek'].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
      dtype=object)

In [55]:
df_filter[df_filter['store_id']==257406274356679]

,store_id,status,timestamp_utc,timezone_str,Date,timestamp_timezone,start_time_local,end_time_local,DayOfWeek,uptime_last_hour,downtime_last_hour
6607,257406274356679,active,2023-01-20 06:03:37.183544 UTC,America/New_York,2023-01-20,2023-01-20 02:03:37.183544,00:00:00,23:59:00,Friday,57,3
6608,257406274356679,active,2023-01-21 05:04:21.810953 UTC,America/New_York,2023-01-21,2023-01-21 01:04:21.810953,00:00:00,23:59:00,Saturday,56,4
7689,257406274356679,active,2023-01-24 18:03:35.955761 UTC,America/New_York,2023-01-24,2023-01-24 14:03:35.955761,00:00:00,23:59:00,Tuesday,57,3
9635,257406274356679,active,2023-01-19 15:09:43.353002 UTC,America/New_York,2023-01-19,2023-01-19 11:09:43.353002,00:00:00,23:59:00,Thursday,51,9
16322,257406274356679,active,2023-01-18 14:10:34.598488 UTC,America/New_York,2023-01-18,2023-01-18 10:10:34.598488,00:00:00,23:59:00,Wednesday,50,10
...,...,...,...,...,...,...,...,...,...,...,...
628701,257406274356679,active,2023-01-22 12:01:19.723014 UTC,America/New_York,2023-01-22,2023-01-22 08:01:19.723014,00:00:00,23:59:00,Sunday,59,1
638382,257406274356679,active,2023-01-19 11:06:14.21261 UTC,America/New_York,2023-01-19,2023-01-19 07:06:14.212610,00:00:00,23:59:00,Thursday,54,6
639812,257406274356679,active,2023-01-18 17:25:19.066889 UTC,America/New_York,2023-01-18,2023-01-18 13:25:19.066889,00:00:00,23:59:00,Wednesday,35,25
641691,257406274356679,active,2023-01-19 02:06:54.02339 UTC,America/New_York,2023-01-19,2023-01-18 22:06:54.023390,00:00:00,23:59:00,Thursday,54,6


In [56]:
df_filter.head()

,store_id,status,timestamp_utc,timezone_str,Date,timestamp_timezone,start_time_local,end_time_local,DayOfWeek,uptime_last_hour,downtime_last_hour
0,3739729523266121270,active,2023-01-24 09:08:23.138922 UTC,America/New_York,2023-01-24,2023-01-24 05:08:23.138922,00:00:00,23:59:59,Tuesday,52,8
1,4430370444829587327,active,2023-01-24 09:09:37.456563 UTC,America/Los_Angeles,2023-01-24,2023-01-24 02:09:37.456563,00:00:00,04:45:00,Tuesday,51,9
2,5627904252074297008,active,2023-01-24 14:01:52.138196 UTC,America/New_York,2023-01-24,2023-01-24 10:01:52.138196,05:30:00,23:59:59,Tuesday,59,1
3,605803287923472894,active,2023-01-24 05:58:22.253512 UTC,America/Los_Angeles,2023-01-24,2023-01-23 22:58:22.253512,10:00:00,23:59:59,Tuesday,2,58
4,5693932324957995427,active,2023-01-24 08:09:45.795033 UTC,America/Chicago,2023-01-24,2023-01-24 03:09:45.795033,00:00:00,23:59:59,Tuesday,51,9


In [57]:
df_filter.to_csv('df_filter.csv', index=False)

In [58]:
df_filter[(df_filter['store_id']==257406274356679) & (df_filter['Date']=='2023-01-20')]

,store_id,status,timestamp_utc,timezone_str,Date,timestamp_timezone,start_time_local,end_time_local,DayOfWeek,uptime_last_hour,downtime_last_hour
6607,257406274356679,active,2023-01-20 06:03:37.183544 UTC,America/New_York,2023-01-20,2023-01-20 02:03:37.183544,00:00:00,23:59:00,Friday,57,3
21987,257406274356679,active,2023-01-20 19:01:28.493827 UTC,America/New_York,2023-01-20,2023-01-20 15:01:28.493827,00:00:00,23:59:00,Friday,59,1
111606,257406274356679,active,2023-01-20 15:00:51.473358 UTC,America/New_York,2023-01-20,2023-01-20 11:00:51.473358,00:00:00,23:59:00,Friday,60,0
145382,257406274356679,active,2023-01-20 23:00:54.243707 UTC,America/New_York,2023-01-20,2023-01-20 19:00:54.243707,00:00:00,23:59:00,Friday,60,0
229937,257406274356679,active,2023-01-20 18:00:52.434722 UTC,America/New_York,2023-01-20,2023-01-20 14:00:52.434722,00:00:00,23:59:00,Friday,60,0
243522,257406274356679,active,2023-01-20 12:01:21.962726 UTC,America/New_York,2023-01-20,2023-01-20 08:01:21.962726,00:00:00,23:59:00,Friday,59,1
262008,257406274356679,active,2023-01-20 09:00:53.649182 UTC,America/New_York,2023-01-20,2023-01-20 05:00:53.649182,00:00:00,23:59:00,Friday,60,0
265345,257406274356679,active,2023-01-20 08:02:34.7341 UTC,America/New_York,2023-01-20,2023-01-20 04:02:34.734100,00:00:00,23:59:00,Friday,58,2
272048,257406274356679,active,2023-01-20 14:00:43.638713 UTC,America/New_York,2023-01-20,2023-01-20 10:00:43.638713,00:00:00,23:59:00,Friday,60,0
281947,257406274356679,active,2023-01-20 20:03:48.983693 UTC,America/New_York,2023-01-20,2023-01-20 16:03:48.983693,00:00:00,23:59:00,Friday,57,3


In [59]:
query = ''' 
    SELECT df_filter.store_id, df_filter.DayOfWeek,df_filter.Date,df_filter.timestamp_timezone , 
    SUM(df_filter.uptime_last_hour) AS uptime_last_day,
    strftime('%H:%M:%S', df_filter.start_time_local) AS start_time_local,
    strftime('%H:%M:%S', df_filter.end_time_local) AS end_time_local,
    (julianday(df_filter.end_time_local) - julianday(df_filter.start_time_local)) * 1440 AS total_time
    FROM df_filter
    GROUP BY df_filter.store_id, df_filter.DayOfWeek, df_filter.Date
''' 
df_filter2 = pd.read_sql_query(query, conn)
df_filter2.head(7)

,store_id,DayOfWeek,Date,timestamp_timezone,uptime_last_day,start_time_local,end_time_local,total_time
0,257406274356679,Friday,2023-01-20,2023-01-20 02:03:37.183544,1092,00:00:00,23:59:00,1439.0
1,257406274356679,Saturday,2023-01-21,2023-01-21 01:04:21.810953,1031,00:00:00,23:59:00,1439.0
2,257406274356679,Sunday,2023-01-22,2023-01-21 23:03:22.016915,406,00:00:00,23:59:00,1439.0
3,257406274356679,Thursday,2023-01-19,2023-01-19 11:09:43.353002,837,00:00:00,23:59:00,1439.0
4,257406274356679,Tuesday,2023-01-24,2023-01-24 14:03:35.955761,1028,00:00:00,23:59:00,1439.0
5,257406274356679,Wednesday,2023-01-18,2023-01-18 10:10:34.598488,415,00:00:00,23:59:00,1439.0
6,257406274356679,Wednesday,2023-01-25,2023-01-24 21:03:11.288352,1084,00:00:00,23:59:00,1439.0


In [60]:
df_filter2['downtime_last_day']=df_filter2['total_time']-df_filter2['uptime_last_day']
df_filter2.head()

,store_id,DayOfWeek,Date,timestamp_timezone,uptime_last_day,start_time_local,end_time_local,total_time,downtime_last_day
0,257406274356679,Friday,2023-01-20,2023-01-20 02:03:37.183544,1092,00:00:00,23:59:00,1439.0,347.0
1,257406274356679,Saturday,2023-01-21,2023-01-21 01:04:21.810953,1031,00:00:00,23:59:00,1439.0,408.0
2,257406274356679,Sunday,2023-01-22,2023-01-21 23:03:22.016915,406,00:00:00,23:59:00,1439.0,1033.0
3,257406274356679,Thursday,2023-01-19,2023-01-19 11:09:43.353002,837,00:00:00,23:59:00,1439.0,602.0
4,257406274356679,Tuesday,2023-01-24,2023-01-24 14:03:35.955761,1028,00:00:00,23:59:00,1439.0,411.0


In [61]:
df_filter2.head(7)

,store_id,DayOfWeek,Date,timestamp_timezone,uptime_last_day,start_time_local,end_time_local,total_time,downtime_last_day
0,257406274356679,Friday,2023-01-20,2023-01-20 02:03:37.183544,1092,00:00:00,23:59:00,1439.0,347.0
1,257406274356679,Saturday,2023-01-21,2023-01-21 01:04:21.810953,1031,00:00:00,23:59:00,1439.0,408.0
2,257406274356679,Sunday,2023-01-22,2023-01-21 23:03:22.016915,406,00:00:00,23:59:00,1439.0,1033.0
3,257406274356679,Thursday,2023-01-19,2023-01-19 11:09:43.353002,837,00:00:00,23:59:00,1439.0,602.0
4,257406274356679,Tuesday,2023-01-24,2023-01-24 14:03:35.955761,1028,00:00:00,23:59:00,1439.0,411.0
5,257406274356679,Wednesday,2023-01-18,2023-01-18 10:10:34.598488,415,00:00:00,23:59:00,1439.0,1024.0
6,257406274356679,Wednesday,2023-01-25,2023-01-24 21:03:11.288352,1084,00:00:00,23:59:00,1439.0,355.0


In [62]:
df_filter2.shape

(57624, 9)

In [63]:
df_filter2.to_sql('df_filter2', conn, if_exists='replace', index=False)

57624

In [64]:
query='''
    SELECT store_id, SUM(uptime_last_day) AS uptime_last_week, 
    CAST(ROUND(SUM(downtime_last_day),0)AS INTEGER) AS downtime_last_week,
    CAST(ROUND(SUM(total_time),0) AS INTEGER) AS total_time_last_week
    FROM df_filter2
    GROUP BY store_id
'''
df_filter3 = pd.read_sql_query(query, conn)
df_filter3.head(7)

,store_id,uptime_last_week,downtime_last_week,total_time_last_week
0,257406274356679,5893,4180,10073
1,309081616338296,5597,4033,9630
2,2570905277901393,8368,266,8634
3,3225796639170927,5520,4553,10073
4,4518233183474396,1532,1588,3120
5,4647420561423097,1223,1177,2400
6,5256638191399657,2898,2712,5610


In [65]:
df_filter3.to_sql('df_filter3', conn, if_exists='replace', index=False)

9343

In [66]:
query='''
    SELECT df_filter2.store_id,df_filter2.DayOfWeek, df_filter2.timestamp_timezone , df_filter2.uptime_last_day, df_filter2.downtime_last_day, df_filter3.uptime_last_week, df_filter3.downtime_last_week
    FROM df_filter2
    LEFT JOIN df_filter3
    ON df_filter2.store_id = df_filter3.store_id 
'''
df_filter4 = pd.read_sql_query(query, conn)
df_filter4.head(7)

,store_id,DayOfWeek,timestamp_timezone,uptime_last_day,downtime_last_day,uptime_last_week,downtime_last_week
0,257406274356679,Friday,2023-01-20 02:03:37.183544,1092,347.0,5893,4180
1,257406274356679,Saturday,2023-01-21 01:04:21.810953,1031,408.0,5893,4180
2,257406274356679,Sunday,2023-01-21 23:03:22.016915,406,1033.0,5893,4180
3,257406274356679,Thursday,2023-01-19 11:09:43.353002,837,602.0,5893,4180
4,257406274356679,Tuesday,2023-01-24 14:03:35.955761,1028,411.0,5893,4180
5,257406274356679,Wednesday,2023-01-18 10:10:34.598488,415,1024.0,5893,4180
6,257406274356679,Wednesday,2023-01-24 21:03:11.288352,1084,355.0,5893,4180


In [67]:
df_filter4.shape

(57624, 7)

In [68]:
df_filter4.to_sql('df_filter4', conn, if_exists='replace', index=False)

57624

In [69]:
df_filter4.head()

,store_id,DayOfWeek,timestamp_timezone,uptime_last_day,downtime_last_day,uptime_last_week,downtime_last_week
0,257406274356679,Friday,2023-01-20 02:03:37.183544,1092,347.0,5893,4180
1,257406274356679,Saturday,2023-01-21 01:04:21.810953,1031,408.0,5893,4180
2,257406274356679,Sunday,2023-01-21 23:03:22.016915,406,1033.0,5893,4180
3,257406274356679,Thursday,2023-01-19 11:09:43.353002,837,602.0,5893,4180
4,257406274356679,Tuesday,2023-01-24 14:03:35.955761,1028,411.0,5893,4180


In [70]:
df_filter.head()

,store_id,status,timestamp_utc,timezone_str,Date,timestamp_timezone,start_time_local,end_time_local,DayOfWeek,uptime_last_hour,downtime_last_hour
0,3739729523266121270,active,2023-01-24 09:08:23.138922 UTC,America/New_York,2023-01-24,2023-01-24 05:08:23.138922,00:00:00,23:59:59,Tuesday,52,8
1,4430370444829587327,active,2023-01-24 09:09:37.456563 UTC,America/Los_Angeles,2023-01-24,2023-01-24 02:09:37.456563,00:00:00,04:45:00,Tuesday,51,9
2,5627904252074297008,active,2023-01-24 14:01:52.138196 UTC,America/New_York,2023-01-24,2023-01-24 10:01:52.138196,05:30:00,23:59:59,Tuesday,59,1
3,605803287923472894,active,2023-01-24 05:58:22.253512 UTC,America/Los_Angeles,2023-01-24,2023-01-23 22:58:22.253512,10:00:00,23:59:59,Tuesday,2,58
4,5693932324957995427,active,2023-01-24 08:09:45.795033 UTC,America/Chicago,2023-01-24,2023-01-24 03:09:45.795033,00:00:00,23:59:59,Tuesday,51,9


In [77]:
query='''
    SELECT df_filter.store_id,df_filter.DayOfWeek, df_filter.timestamp_timezone, df_filter.uptime_last_hour, df_filter.downtime_last_hour, ROUND((df_filter4.uptime_last_day)/60.0,2) AS uptime_last_day, ROUND((df_filter4.downtime_last_day)/60.0,2) AS downtime_last_day, ROUND((df_filter4.uptime_last_week)/60.0,2) AS uptime_last_week, ROUND((df_filter4.downtime_last_week)/60.0,2) AS downtime_last_week
    FROM df_filter
    LEFT JOIN df_filter4
    ON df_filter.store_id = df_filter4.store_id 
'''
df_filter5 = pd.read_sql_query(query, conn)
df_filter5.head(7)

,store_id,DayOfWeek,timestamp_timezone,uptime_last_hour,downtime_last_hour,uptime_last_day,downtime_last_day,uptime_last_week,downtime_last_week
0,3739729523266121270,Tuesday,2023-01-24 05:08:23.138922,52,8,3.0,20.83,95.0,72.0
1,3739729523266121270,Tuesday,2023-01-24 05:08:23.138922,52,8,6.0,17.62,95.0,72.0
2,3739729523266121270,Tuesday,2023-01-24 05:08:23.138922,52,8,15.0,8.07,95.0,72.0
3,3739729523266121270,Tuesday,2023-01-24 05:08:23.138922,52,8,16.0,7.90,95.0,72.0
4,3739729523266121270,Tuesday,2023-01-24 05:08:23.138922,52,8,17.0,6.83,95.0,72.0
5,3739729523266121270,Tuesday,2023-01-24 05:08:23.138922,52,8,18.0,5.88,95.0,72.0
6,3739729523266121270,Tuesday,2023-01-24 05:08:23.138922,52,8,18.0,5.53,95.0,72.0


In [79]:
df_filter5['downtime_last_day']=abs(df_filter5['downtime_last_day'])
df_filter5['downtime_last_week']=abs(df_filter5['downtime_last_week'])

In [80]:
df_filter5.to_csv('df_filter5.csv', index=False)

In [121]:
# query = ''' 
#     SELECT df_filter.store_id, df_filter.DayOfWeek,df_filter.Date,df_filter.timestamp_timezone , 
#     SUM(df_filter.uptime_last_hour) AS uptime_last_day
#     FROM df_filter
#     GROUP BY df_filter.store_id, df_filter.DayOfWeek, df_filter.Date
# ''' 
# df_filter2 = pd.read_sql_query(query, conn)
# df_filter2.head(7)

In [120]:
df_filter2.to_csv('df_filter2.csv', index=False)

In [69]:
df_filter2['DayOfWeek'].unique()

array(['Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'Tuesday'],
      dtype=object)

In [71]:
df_filter2.head()

,store_id,DayOfWeek,timestamp_timezone,uptime_last_hour,downtime_last_hour,uptime_last_day,downtime_last_day
0,257406274356679,Wednesday,2023-01-18 09:31:37.537599,29,31,29,31
1,257406274356679,Wednesday,2023-01-18 10:10:34.598488,50,10,50,10
2,257406274356679,Wednesday,2023-01-18 11:36:51.841037,24,36,24,36
3,257406274356679,Wednesday,2023-01-18 12:06:11.775275,54,6,54,6
4,257406274356679,Wednesday,2023-01-18 13:25:19.066889,35,25,35,25


In [72]:
df_filter2.shape

(645564, 7)

In [75]:
df_Menu_hours.shape

(86198, 5)